In [6]:
# Download the required packages
!pip install -r requirements.txt

  Obtaining dependency information for networkx==2.8.6 from https://files.pythonhosted.org/packages/be/25/5b0fc262a2f2d7d11c22cb7785edf2befc756ae076b383034e79e255eb11/networkx-2.8.6-py3-none-any.whl.metadata
  Obtaining dependency information for numpy==1.23.3 from https://files.pythonhosted.org/packages/b1/84/0af94541d21dd2d403377209f462b6b463dc4ba15158776285f1af2132ac/numpy-1.23.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 18.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for streamlit==1.12.2 from https://files.pythonhosted.org/packages/46/73/0b497b5dbf99079c7f70dab726798945390477f1fd3a6b036188e1a3b90c/streamlit-1.12.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for jsonpickle>=1.4.1 from https://files.pythonhosted.o

In [2]:
# Import all the necessary libraries
import numpy as np
import pandas as pd
import streamlit as st
import streamlit.components.v1 as components
import networkx as nx
import snowflake.connector
from pyvis.network import Network
import matplotlib.pyplot as plt

In [6]:
# Snowflake Connection
conn = snowflake.connector.connect(
    user = st.secrets["USER"],
    password = st.secrets["PASSWORD"],
    account = st.secrets["ACCOUNT"],
    warehouse = st.secrets["WAREHOUSE"],
    database = st.secrets["DATABASE"],
    schema = st.secrets["SCHEMA"]
)

# Query to get the data
query = """
WITH transaction_paths AS (
    -- Step 1: Start with all transactions as starting points
    SELECT 
        sender_account AS start_account,
        beneficiary_account AS current_account,
        transaction_amount AS start_amount,
        transaction_amount AS current_amount,
        transaction_date AS start_date,
        transaction_date AS current_date,
        ARRAY_CONSTRUCT(sender_account, beneficiary_account) AS account_path,
        ARRAY_CONSTRUCT(transaction_date) AS date_path,
        ARRAY_CONSTRUCT(transaction_amount) AS amount_path,
        ARRAY_CONSTRUCT(transaction_id) AS id_path,
        1 AS depth
    FROM "CYGNUS"."PUBLIC"."TRANSACTIONS"

    UNION ALL

    -- Step 2: Extend paths by finding valid next transactions
    SELECT 
        tp.start_account,
        t.beneficiary_account,
        tp.start_amount,
        t.transaction_amount,
        tp.start_date,
        t.transaction_date,
        ARRAY_APPEND(tp.account_path, t.beneficiary_account),
        ARRAY_APPEND(tp.date_path, t.transaction_date),
        ARRAY_APPEND(tp.amount_path, t.transaction_amount),
        ARRAY_APPEND(tp.id_path, t.transaction_id),
        tp.depth + 1
    FROM transaction_paths tp,
         LATERAL (
            SELECT *
            FROM "CYGNUS"."PUBLIC"."TRANSACTIONS" t
            WHERE tp.current_account = t.sender_account
            AND t.transaction_date > tp.current_date
            AND t.transaction_amount BETWEEN tp.current_amount * 0.9 AND tp.current_amount * 1.1
            AND NOT ARRAY_CONTAINS(tp.account_path, ARRAY_CONSTRUCT(t.beneficiary_account)) -- Prevent cycles before completing loop
        ) t
)
SELECT account_path, date_path, amount_path, id_path, depth
FROM transaction_paths
WHERE current_account = start_account  -- Ensure loop completion
AND depth > 2  -- At least one intermediate transaction
ORDER BY start_date
"""

# Execute the query
cur = conn.cursor()
cur.execute(query)

# Fetch the data
rows = cur.fetchall()

# Create a dataframe
df = pd.DataFrame(rows, columns=[x[0] for x in cur.description])

# Close the cursor and connection
cur.close()
conn.close()

2025-02-07 20:52:58.031 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 3.13.2, Python Version: 3.11.5, Platform: macOS-15.3-arm64-arm-64bit
2025-02-07 20:52:58.032 INFO    snowflake.connector.connection: Connecting to GLOBAL Snowflake domain
2025-02-07 20:52:58.032 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2025-02-07 20:52:58.855 INFO    snowflake.connector.connection: closed
2025-02-07 20:52:58.903 INFO    snowflake.connector.connection: No async queries seem to be running, deleting session


In [11]:
df

,transaction_id,sender_account,beneficiary_account,transaction_date,transaction_amount
index,,,,,
0,9007,Account_493,Account_604,2024-01-05,597.79
0,5533,Account_604,Account_42,2024-06-27,560.92
0,4624,Account_42,Account_493,2024-11-24,518.29
1,26,Account_548,Account_711,2024-01-08,600.01
1,6455,Account_711,Account_471,2024-02-04,630.80
1,4909,Account_471,Account_617,2024-04-02,662.22
1,9886,Account_617,Account_548,2024-12-02,636.55
2,3174,Account_359,Account_387,2024-01-31,954.55
2,2946,Account_387,Account_594,2024-05-08,936.17


In [8]:
# Process the output
df.columns = ['path', 'date_path', 'amount_path', 'id_path', 'depth']

# Clean path, date_path, and amount_path columns by removing \n from array
df['path'] = df['path'].apply(lambda x: eval(x))
df['date_path'] = df['date_path'].apply(lambda x: eval(x))
df['amount_path'] = df['amount_path'].apply(lambda x: eval(x))
df['id_path'] = df['id_path'].apply(lambda x: eval(x))

In [10]:
# Expand lists into rows while keeping the original index
rows = []
for idx, row in df.iterrows():
    for i in range(len(row["date_path"])):
        rows.append([
            idx,  # Preserve original index
            row["id_path"][i],
            row["path"][i],
            row["path"][i + 1],
            row["date_path"][i],
            row["amount_path"][i]
        ])

# Create new DataFrame
df = pd.DataFrame(rows, columns=["index", "transaction_id", "sender_account", "beneficiary_account", "transaction_date", "transaction_amount"]).set_index("index")

In [14]:
df

# calculate %diff from previous row if index is same
df['amount_diff'] = df.groupby('index')['transaction_amount'].pct_change()*100

df

,transaction_id,sender_account,beneficiary_account,transaction_date,transaction_amount,amount_diff
index,,,,,,
0,9007,Account_493,Account_604,2024-01-05,597.79,NaN
0,5533,Account_604,Account_42,2024-06-27,560.92,-6.167718
0,4624,Account_42,Account_493,2024-11-24,518.29,-7.600014
1,26,Account_548,Account_711,2024-01-08,600.01,NaN
1,6455,Account_711,Account_471,2024-02-04,630.80,5.131581
1,4909,Account_471,Account_617,2024-04-02,662.22,4.980977
1,9886,Account_617,Account_548,2024-12-02,636.55,-3.876355
2,3174,Account_359,Account_387,2024-01-31,954.55,NaN
2,2946,Account_387,Account_594,2024-05-08,936.17,-1.925515


In [ ]:
import streamlit as st
import networkx as nx
from pyvis.network import Network
import pandas as pd
import streamlit.components.v1 as components

# # Sample dataframe (replace with actual df)
# df = pd.DataFrame({
#     'account_send': ['A', 'B', 'C', 'D', 'E'],
#     'account_receive': ['B', 'C', 'D', 'E', 'A'],
#     'amount': [100, 200, 150, 300, 250],
#     'date': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05']
# })

# Set header title
st.title('Network Graph Visualization of Transaction Loops')

# Define list of selection options
suspicious_accounts = df.account_send.unique()
suspicious_accounts.sort()

# Implement selection box for account visualization
selected_account = st.selectbox('Select account to visualize', suspicious_accounts)

if selected_account:
    # Filter data for selected account
    df_select = df[(df.account_send == selected_account) | (df.account_receive == selected_account)]

    # Create a directed graph
    G = nx.DiGraph()

    # Add nodes and edges with attributes
    for _, row in df_select.iterrows():
        sender_color = "#4CAF50"  # Green for intermediary accounts
        receiver_color = "#2196F3"  # Blue for origination accounts

        G.add_node(row["account_send"], label=str(row["account_send"]), color=sender_color)
        G.add_node(row["account_receive"], label=str(row["account_receive"]), color=receiver_color)
        G.add_edge(
            row["account_send"], row["account_receive"],
            title=f"Amount: ${row['amount']}<br>Date: {row['date']}",
            label=f"${row['amount']}\n{row['date']}",
            color="#FF9800"
        )

    # Create PyVis Network
    account_network = Network(height="500px", width="100%", directed=True, bgcolor="#222222", font_color="white")

    # Convert NetworkX graph to PyVis
    account_network.from_nx(G)

    # Improve layout with physics settings
    account_network.repulsion(node_distance=300, central_gravity=0.3, spring_length=100, spring_strength=0.1, damping=0.9)

    # Ensure edge labels are always visible
    for edge in account_network.edges:
        edge["font"] = {"size": 12, "color": "white", "background": "black"}

    # Add legend manually
    account_network.add_node("Origination Account", color="#2196F3", shape="dot", size=10)
    account_network.add_node("Intermediary Accounts", color="#4CAF50", shape="dot", size=10)

    # Save and read graph as HTML file
    path = '/tmp' if 'tmp' in dir() else '/html_files'
    account_network.save_graph(f'{path}/pyvis_graph.html')
    HtmlFile = open(f'{path}/pyvis_graph.html', 'r', encoding='utf-8')

    # Display graph in Streamlit
    components.html(HtmlFile.read(), height=500)

    # Show filtered dataframe below the graph
    st.subheader(f"Transaction Details for Account: {selected_account}")
    st.dataframe(df_select)

In [8]:
# Set header title
st.title('Network Graph Visualization of Transaction Loops')

# Define list of selection options
suspicious_accounts = df.account_send.unique()
suspicious_accounts.sort()

# Implement multiselect dropdown menu for option selection (returns a list)
selected_account = st.selectbox('Select account to visualize', suspicious_accounts)

# Set info message on initial site load
if selected_account:
    st.text('Choose an account to start')

# Create network graph when user selects >= 1 item
else:
    df_select = df[df.index == df[df.account_send == selected_account].index[0]]

    # Create a directed graph
    G = nx.DiGraph()

    # Add nodes and edges with attributes
    for _, row in df_select.iterrows():
        G.add_node(row["account_send"], label=str(row["account_send"]), color="#4CAF50")  # Green sender
        G.add_node(row["account_receive"], label=str(row["account_receive"]), color="#2196F3")  # Blue receiver
        G.add_edge(
            row["account_send"], row["account_receive"],
            title=f"Amount: ${row['amount']}\nDate: {row['date']}",  # Tooltip when hovering
            label=f"${row['amount']}\n{row['date']}",  # Visible label
            color="#FF9800"  # Orange edges
        )

    # Create PyVis Network
    account_network = Network(height="500px", width="100%", directed=True, bgcolor="#222222", font_color="white")

    # Convert NetworkX graph to PyVis
    account_network.from_nx(G)

    # Improve layout with physics settings
    account_network.repulsion(node_distance=300, central_gravity=0.3, spring_length=100, spring_strength=0.1, damping=0.9)

    # Force edge labels to show
    for edge in account_network.edges:
        edge["font"] = {"size": 12, "color": "white", "background": "black"}

    # Save and read graph as HTML file (on Streamlit Sharing)
    try:
        path = '/tmp'
        account_network.save_graph(f'{path}/pyvis_graph.html')
        HtmlFile = open(f'{path}/pyvis_graph.html', 'r', encoding='utf-8')

    # Save and read graph as HTML file (locally)
    except:
        path = '/html_files'
        account_network.save_graph(f'{path}/pyvis_graph.html')
        HtmlFile = open(f'{path}/pyvis_graph.html', 'r', encoding='utf-8')

    # Load HTML file in HTML component for display on Streamlit page
    components.html(HtmlFile.read(), height=435)

2025-02-07 19:27:55.828 
  command:

    streamlit run /Users/arnavgarg/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
# # Sample data
# df = df.copy()

# # Initialize directed graph
# G = nx.DiGraph()

# # Add edges to the graph
# for row in range(3):
#     path, dates, amounts = df['path'][row], df['date_path'][row], df['amount_path'][row]
#     for i in range(len(dates)):
#         G.add_edge(path[i], path[i + 1], date=dates[i], amount=amounts[i])

# # # Draw the graph
# # pos = nx.circular_layout(G)  # Layout
# # plt.figure(figsize=(6, 6))
# # nx.draw(G, pos, with_labels=True, node_color="lightblue", edge_color="gray", node_size=2000, font_size=10)

# # # Add edge labels (date and amount)
# # edge_labels = {(u, v): f"{d['date']}\n${d['amount']}" for u, v, d in G.edges(data=True)}
# # nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

# # plt.title("Transaction Loops")
# # plt.show()